In [1]:
import pandas as pd
import re
import multiprocessing

In [ ]:
# yahoo_data=pd.read_csv('yahoo.csv',names=['name','passwd'])
# csdn_data=pd.read_csv('csdn.csv',error_bad_lines=False,names=['name','email','passwd'])

In [2]:
# 删除所有有空值的行
# csdn_data.dropna(inplace=True)

In [ ]:
# lds 是一个全局变量，存储<密码格式，密码格式出现的次数>，这一个键值对
#char_dic 也是一个全局变量，他存储每一个字符出现的次数，这一个键值对
# lds_csdn={}
# lds_yahoo={}
# char_dic_csdn={}
# char_dic_yahoo={}

In [2]:
def passwd_to_lds_cols(passwd,char_dic):
    '''
       处理每一个密码，得到他的格式，然将这个格式封装成list
       如密码：KIC43dk6!
       经过处理后返回结果为：['l', 'l', 'l', 'd', 'd', 'l', 'l', 'd', 's']
    '''
    l_count=0
    d_count=0
    s_count=0

    re_l='[a-zA-Z]'
    re_d='[0-9]'
    # res={}
    col=[]
    for i in range (len(passwd)):
        temp=passwd[i]
        # print[temp]
        if re.match(re_l,temp):
            l_count+=1
            col.append('l')

           
        elif re.match(re_d,temp):
            d_count+=1
            col.append('d')
        else:
            s_count+=1
            col.append('s')

        # 统计每一个字符出现的次数
        if passwd[i] not in char_dic:
            char_dic[passwd[i]]=1
        else:
            char_dic[passwd[i]]+=1
    return col


In [3]:
def lds_cols_format(lds_cols):
    '''
        这个是将passwd_to_lds_cols处理密码得到的list进行格式化，得到一个密码格式的字符串
        如输入的：['l', 'l', 'l', 'd', 'd', 'l', 'l', 'd', 's']
        经过处理后为：l3d2l2d1s1
    '''
    res=[]
    count=0
    lds_temp=lds_cols.copy()
    # 最后一个处理不到，所以添加一个空格，来时处理到源字符串的最后一个
    lds_temp.append(' ')
    first=lds_temp[0]
    for i in range(len(lds_temp)):
        if lds_temp[i]==first:
            count+=1
            first=lds_temp[i]
            continue
        else:
            if i==len(lds_temp):

                pass
            temp=first+str(count)
            res.append(temp)
            first=lds_temp[i]

            count=1
    # 数据拼接
    r=''.join(res)

            
    return r

In [4]:
# passwd_csdn=csdn_data.passwd
# passwd_yahoo=yahoo_data.pa


In [4]:
def precoss_passwd(passwd,lds,char_dic):
    # lenght 这个变量是可有可无的
    length=len(passwd)
    for pwd in passwd:
        temp=passwd_to_lds_cols(pwd,char_dic)
        temp1=lds_cols_format(temp)
        if temp1 not in lds:
            lds[temp1]=1
        else:
            lds[temp1]+=1
    return length

In [7]:
def get_top10_of_dict(dict):
    '''
    获取前10的信息
    '''
    lds_sorted=sorted(dict.items(),key=lambda x :x[1],reverse=True)
    return lds_sorted[:10]

In [12]:
def main_proc():

    print('start')
    yahoo_data=pd.read_csv('yahoo.csv',names=['name','passwd',0,1,2,3])
    csdn_data=pd.read_csv('csdn.csv',error_bad_lines=False,names=['name','email','passwd'])
    print("load data")
    # 对数据进行处理
    yahoo_data.drop(columns=[0,1,2,3],axis=1,inplace=True)
    csdn_data.dropna(inplace=True)
    yahoo_data.dropna(inplace=True)


    # 用来保存，数据集的密码格式和单词频率统计信息 
    lds_csdn={}
    lds_yahoo={}
    char_dic_csdn={}
    char_dic_yahoo={}
   
    passwd_csdn=csdn_data.passwd
    passwd_yahoo=yahoo_data.passwd
    print(passwd_yahoo.head())
    print('handle passwd')
    # 开两个进程 来处理含不同的数据源。
    l1= precoss_passwd(passwd_csdn,lds_csdn,char_dic_csdn)
    # p1=multiprocessing.Process(target=precoss_passwd,args=(passwd_csdn,lds_csdn,char_dic_csdn))
    l2 =precoss_passwd(passwd_yahoo,lds_yahoo,char_dic_yahoo)
    # p2=multiprocessing.Process(target=precoss_passwd,args=(passwd_yahoo,lds_yahoo,char_dic_yahoo))
   
    # p1.start()
    # p2.start()
    
    # p1.join()
    # p2.join()

    print(l1)
    print(l2)
    # 得到经过处理过过后的排名前10的密码格式
    lds_csdn_top10=get_top10_of_dict(lds_csdn)
    lds_yahoo_top10=get_top10_of_dict(lds_yahoo)

    print(lds_csdn_top10)
    print('==============================')
    print(lds_yahoo_top10)

#    对lds进行处理，排序
    

In [13]:
%%time
main_proc()

start


D:\Anaconda3\lib\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


load data
0    @fl!pm0de@
1          pass
2       steveol
3        chotzi
4        lb2512
Name: passwd, dtype: object
handle passwd
6298242
442807
[('d8', 1352319), ('d9', 704226), ('l8', 307253), ('d11', 268536), ('d10', 235464), ('l3d6', 184199), ('l9', 166924), ('l2d6', 137564), ('l10', 129103), ('d12', 103780)]
[('l6', 42239), ('l8', 34286), ('l7', 30253), ('l6d2', 22290), ('l9', 16887), ('l8d2', 13339), ('l7d1', 11956), ('l7d2', 11293), ('l10', 10819), ('d6', 10075)]
Wall time: 3min 11s


In [ ]:
# for pwd in passwd:
#     temp=passwd_to_lds_cols(pwd)
#     temp1=lds_cols_format(temp)
#     if temp1 not in lds:
#         lds[temp1]=1
#     else:
#         lds[temp1]+=1

In [ ]:
# def format_dic_count(lds_formated_list):
#     '''
#         统计每一个密码格式串出现的次数
#     '''
#     for formated in lds_formated_list:
#         if formated not in lds:
#             lds[formated]=1
#         else:
#             lds[formated]+=1
    



In [3]:
import pandas as pd
yahoo_data=pd.read_csv('yahoo.csv',names=['name','passwd',0,1,2,3])
csdn_data=pd.read_csv('csdn.csv',error_bad_lines=False,names=['name','email','passwd'])
print("load data")
# 对数据进行处理
yahoo_data.drop(columns=[0,1,2,3],axis=1,inplace=True)
csdn_data.dropna(inplace=True)
yahoo_data.dropna(inplace=True)
csdn_data.to_csv('handled_scdn.csv',index=False)
yahoo_data.to_csv('handled_yahoo.csv',index=False)


D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


load data
